In [261]:
import numpy as np
import pandas as pd
import pydicom
import glob
import os
import shutil

In [2]:
dir_path = '/data/jiahong/data/FDG_PET/'

In [3]:
case_id_dir_list = glob.glob(dir_path+'*')
case_id_list_from_path = [path.split('/')[-1] for path in case_id_dir_list if os.path.isdir(path)]
# print(case_id_list_from_path)

In [4]:
case_id_list_case_1 = [case_id for case_id in case_id_list_from_path if 'case' in case_id and '_' in case_id]
case_id_list_case_1 = sorted(case_id_list_case_1, key=lambda case_id: int(case_id.split('_')[1]))

case_id_list_case_2 = [case_id for case_id in case_id_list_from_path if 'case' in case_id and '-' in case_id]
case_id_list_case_2 = sorted(case_id_list_case_2, key=lambda case_id: int(case_id.split('-')[1]))

case_id_list_fdg = [case_id for case_id in case_id_list_from_path if 'Fdg' in case_id or 'Anonymized' in case_id]
case_id_list_fdg = sorted(case_id_list_fdg, key=lambda case_id: int(case_id.split('_')[-1]))

def is_number(case_id):
    if 'case' in case_id:
        return False
    if 'Fdg_Stanford' in case_id:
        return False
    if 'Anonymized' in case_id:
        return False
    return True

case_id_list_num = [case_id for case_id in case_id_list_from_path if is_number(case_id)]
case_id_list_num = sorted(case_id_list_num, key=lambda case_id: int(case_id.split('_')[0]))

case_id_list = case_id_list_case_1 + case_id_list_case_2 + case_id_list_fdg + case_id_list_num
print(len(case_id_list))

280


In [5]:
def print_dict(given_dict):
    for key in given_dict:
        print(key)
        print(given_dict[key])

In [21]:
# select all needed contrasts

def is_in(name, check_list):
    for check_str in check_list:
        if check_str in name:
            return True
    return False

# def is_not_in(name, check_list):
#     for check_str in check_list:
#         if check_str in name:
#             return False
#     return True

def is_T1_T1c(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['t1', 'bravo', 'mprage', 'spgr']) and not is_in(ds.SeriesDescription.lower(), ['flair']):
        if is_in(ds.SeriesDescription.lower(), ['+c', '+_c', 'c+', 'post']) and not is_in(ds.SeriesDescription.lower(), ['pre']):
            return 1   # T1c
        elif not is_in(ds.SeriesDescription.lower(), ['+c', '+_c', 'c+', 'post']):
            return 0   # T1
    return -1     # not T1 or T1c

def is_T2_FLAIR(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['flair']) and not is_in(ds.SeriesDescription.lower(), ['t1']):
        return 1
    return -1
    
def is_ASL(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['asl']):
        return 1
    return -1

def is_T2(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['t2']) and not is_in(ds.SeriesDescription.lower(), ['flair']):
        return 1
    return -1

def is_DWI(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['dwi']):
        return 1
    return -1

def is_ADC(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['adc']):
        return 1
    return -1

def is_GRE(ds):
    if 'Modality' in ds:
        if ds.Modality != 'MR':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['gre']):
        return 1
    return -1

def is_PET(ds):
    if 'Modality' in ds:
        if ds.Modality != 'PT':
            return -1
    if is_in(ds.SeriesDescription.lower(), ['mac', 'clear']):
        return 1
    return -1


def select_contrasts(case_path):
    case_dict = {}
    contrast_name_path = {None: 'None'}
    match_contrast_name = {'T1':[], 'T1c':[], 'T2-FLAIR':[], 'ASL':[], 'PET':[], 'T2':[], 'DWI':[], 'GRE':[]}
    for paths in os.walk(case_path):
        if len(paths[-1]) == 0:   # no file, go into subfolders
            continue
        is_dicom = False
        for path in paths[-1]: # check whether dcm files
            if '.dcm' in path:
                is_dicom = True
                break
        if not is_dicom:
            continue

        # save all contrast name and path
        contrast_path = paths[0]
        contrast_name = os.path.basename(paths[0])
        contrast_name_path[contrast_name] = contrast_path
        
        contrast_path = glob.escape(contrast_path)
        pattern = f"{contrast_path}/*"
        if (not os.path.exists(contrast_path)) or len(glob.glob(pattern))==0:
            continue
        
#         print(contrast_path)
        dicom_path = glob.glob(contrast_path+'/*')[0]
        ds = pydicom.filereader.dcmread(dicom_path)
        
        if 'SeriesDescription' not in ds:
#             print('No SeriesDescription', dicom_path)
            continue

        # check T1 / T1c
        is_t1 = is_T1_T1c(ds)
        if is_t1 == 0:
            match_contrast_name['T1'].append(contrast_name)
            continue
        elif is_t1 == 1:
            match_contrast_name['T1c'].append(contrast_name)
            continue

        # check T2-FLAIR
        if is_T2_FLAIR(ds) == 1:
            match_contrast_name['T2-FLAIR'].append(contrast_name)
            continue

        # check ASL
        if is_ASL(ds) == 1:
            match_contrast_name['ASL'].append(contrast_name)
            continue

        # check T2
        if is_T2(ds) == 1:
            match_contrast_name['T2'].append(contrast_name)
            continue

        # check DWI
        if is_DWI(ds) == 1:
            match_contrast_name['DWI'].append(contrast_name)
            continue

        # check GRE
        if is_GRE(ds) == 1:
            match_contrast_name['GRE'].append(contrast_name)
            continue

        # check PET
        if is_PET(ds) == 1:
            match_contrast_name['PET'].append(contrast_name)
            continue

    case_dict['contrast_name_path'] = contrast_name_path
    case_dict['match_contrast_name'] = match_contrast_name
#     print_dict(match_contrast_name)
    
    return case_dict

In [22]:
# select the best needed contrasts

def select_T1_T1c(contrast_name_path, contrast_name_list):
    se_list = []
    gre_list = []
    for contrast_name in contrast_name_list:
        dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
        ds = pydicom.filereader.dcmread(dicom_path)
        if is_in(ds.SeriesDescription.lower(), ['bravo', 'mprage', 'spgr']): # check by description
            gre_list.append(contrast_name)
            continue
        elif is_in(ds.SeriesDescription.lower(), ['spin', 'se']):
            se_list.append(contrast_name)
            continue
        if 'ScanningSequence' in ds:    # check by tag
            if 'GR' in ds.ScanningSequence:
                gre_list.append(contrast_name)
                continue
            elif 'SE' in ds.ScanningSequence:
                se_list.append(contrast_name)
                continue
        if 'EchoTime' in ds:       # check by te and tr
            if ds.EchoTime < 6:
                gre_list.append(contrast_name)
                continue
            else:
                se_list.append(contrast_name)
                continue
#         print(contrast_name_path)
            
    # select the one with smallest slice thickness
    gre_sel = None
    if len(gre_list) > 0:
        thickness_list = []
        thickness_list_else = []
        gre_list_sel = []
        for contrast_name in gre_list:
            dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
            ds = pydicom.filereader.dcmread(dicom_path)
            if int(ds.SeriesNumber) < 100:
                gre_list_sel.append(contrast_name)
                thickness_list.append(float(ds.SliceThickness))
            else:
                thickness_list_else.append(float(ds.SliceThickness))
        if len(gre_list_sel) == 0:
            gre_list_sel = gre_list
            thickness_list = thickness_list_else
        gre_list_sel = np.array(gre_list_sel)
        gre_list_sel = gre_list_sel[np.array(thickness_list)==np.min(thickness_list)]
        for contrast_name in gre_list_sel:
            if 'BRAVO' in contrast_name:
                gre_sel = contrast_name
        if gre_sel == None:
            gre_sel = gre_list_sel[0]
    
    se_sel = None
    if len(se_list) > 0:
        thickness_list = []
        thickness_list_else = []
        se_list_sel = []
        for contrast_name in se_list:
            dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
            ds = pydicom.filereader.dcmread(dicom_path)
            if int(ds.SeriesNumber) < 100:
                se_list_sel.append(contrast_name)
                thickness_list.append(float(ds.SliceThickness))
            else:
                thickness_list_else.append(float(ds.SliceThickness))
        if len(se_list_sel) == 0:
            se_list_sel = se_list
            thickness_list = thickness_list_else
        se_list_sel = np.array(se_list_sel)
        se_list_sel = se_list_sel[np.array(thickness_list)==np.min(thickness_list)]
        se_sel = se_list_sel[0]
    
    return gre_list, se_list, gre_sel, se_sel


def select_T2_FLAIR(contrast_name_path, contrast_name_list):
    se_list = []
    twodim_list = []
    for contrast_name in contrast_name_list:
        dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
        ds = pydicom.filereader.dcmread(dicom_path)
        if 'MRAcquisitionType' in ds:    # remove if 2D
            if ds.MRAcquisitionType == '2D':
                twodim_list.append(contrast_name)
                continue
        if 'ScanningSequence' in ds:    # check by tag
            if 'GR' in ds.ScanningSequence:
                se_list.append(contrast_name)
                continue
        if 'EchoTime' in ds:       # check by te and tr
            if ds.RepetitionTime < 7000:
                se_list.append(contrast_name)
                continue
#         print(contrast_name_path)

    # select the one with smallest slice thickness
    se_sel = None
    if len(se_list) > 0:
        thickness_list = []
        thickness_list_else = []
        se_list_sel = []
        for contrast_name in se_list:
            dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
            ds = pydicom.filereader.dcmread(dicom_path)
            if int(ds.SeriesNumber) < 100:
                se_list_sel.append(contrast_name)
                thickness_list.append(float(ds.SliceThickness))
            else:
                thickness_list_else.append(float(ds.SliceThickness))
        if len(se_list_sel) == 0:
            se_list_sel = se_list
            thickness_list = thickness_list_else
        se_list_sel = np.array(se_list_sel)
        se_list_sel = se_list_sel[np.array(thickness_list)==np.min(thickness_list)]
        se_sel = se_list_sel[0]
    
    twodim_sel = None
    if len(twodim_list) > 0:
        twodim_sel = twodim_list[0]
        
    return se_list, twodim_list, se_sel, twodim_sel

def select_ASL(contrast_name_path, contrast_name_list):
    # usually only two, one is derived from the other
    gre_sel = None
    gre_list_sel = []
    if len(contrast_name_list) > 0:
        for contrast_name in contrast_name_list:
            dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
            ds = pydicom.filereader.dcmread(dicom_path)
            if int(ds.SeriesNumber) < 100:
                gre_list_sel.append(contrast_name)
        gre_sel = gre_list_sel[0]
        
    return gre_list_sel, gre_sel

def select_DWI(contrast_name_path, contrast_name_list):
    # usually only two, one is derived from the other
    gre_sel = None
    gre_list_sel = []
    if len(contrast_name_list) > 0:
        for contrast_name in contrast_name_list:
            dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
            ds = pydicom.filereader.dcmread(dicom_path)
            if int(ds.SeriesNumber) < 100:
                if is_in(ds.SeriesDescription.lower(), ['down']):    # only keep blip up
                    continue
                if is_in(ds.SeriesDescription.lower(), ['ADC']):    # only keep Trace
                    continue
                gre_list_sel.append(contrast_name)
        if len(gre_list_sel) > 0:
            gre_sel = gre_list_sel[0]
        else:
            gre_sel = contrast_name_list[0]
        
    return gre_list_sel, gre_sel

def select_GRE(contrast_name_path, contrast_name_list):
    # usually only two, one is derived from the other
    gre_sel = None
    gre_list_sel = []
    if len(contrast_name_list) > 0:
        for contrast_name in contrast_name_list:
            dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
            ds = pydicom.filereader.dcmread(dicom_path)
            if int(ds.SeriesNumber) < 100:
                gre_list_sel.append(contrast_name)
        if len(gre_list_sel) > 0:
            gre_sel = gre_list_sel[0]
        else:
            gre_sel = contrast_name_list[0]
        
    return gre_list_sel, gre_sel

def select_PET(contrast_name_path, contrast_name_list):
    mac_list = []
    qclear_list = []
    for contrast_name in contrast_name_list:
        dicom_path = glob.glob(contrast_name_path[contrast_name]+'/*')[0]
        ds = pydicom.filereader.dcmread(dicom_path)
        if is_in(ds.SeriesDescription.lower(), ['mac']): # check by description
            mac_list.append(contrast_name)
            continue
        elif is_in(ds.SeriesDescription.lower(), ['clear']):
            qclear_list.append(contrast_name)
            continue
        elif is_in(ds.SeriesDescription.lower(), ['ac']) and not is_in(ds.SeriesDescription.lower(), ['nac']):
            mac_list.append(contrast_name)
            continue
            
    mac_sel = None
    if len(mac_list) > 0:
        mac_sel = mac_list[0]
    
    qclear_sel = None
    if len(qclear_list) > 0:
        qclear_sel = qclear_list[0]
    
    return mac_list, qclear_list, mac_sel, qclear_sel

In [23]:
t1_gre_list = []
t1_se_list = []
t1c_gre_list = []
t1c_se_list = []
t2_flair_gre_list = []
t2_flair_2d_list = []
asl_list = []
dwi_list = []
gre_list = []
pet_mac_list = []
pet_qclear_list = []

t1_gre_path_list = []
t1_se_path_list = []
t1c_gre_path_list = []
t1c_se_path_list = []
t2_flair_gre_path_list = []
t2_flair_2d_path_list = []
asl_path_list = []
dwi_path_list = []
gre_path_list = []
pet_mac_path_list = []
pet_qclear_path_list = []


for case_id in case_id_list:
    print(case_id)
    case_path = os.path.join(dir_path, case_id)
    case_dict = select_contrasts(case_path)
    # T1, T1c     
    _, _, t1_gre_sel, t1_se_sel = select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1'])
    _, _, t1c_gre_sel, t1c_se_sel = select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1c'])
    t1_gre_list.append(t1_gre_sel)
    t1_se_list.append(t1_se_sel)
    t1c_gre_list.append(t1c_gre_sel)
    t1c_se_list.append(t1c_se_sel)
    t1_gre_path_list.append(case_dict['contrast_name_path'][t1_gre_sel])
    t1_se_path_list.append(case_dict['contrast_name_path'][t1_se_sel])
    t1c_gre_path_list.append(case_dict['contrast_name_path'][t1c_gre_sel])
    t1c_se_path_list.append(case_dict['contrast_name_path'][t1c_se_sel])
    
    # T2-FLAIR
    _, _, t2_flair_gre_sel, t2_flair_2d_sel = select_T2_FLAIR(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T2-FLAIR'])
    t2_flair_gre_list.append(t2_flair_gre_sel)
    t2_flair_2d_list.append(t2_flair_2d_sel)
    t2_flair_gre_path_list.append(case_dict['contrast_name_path'][t2_flair_gre_sel])
    t2_flair_2d_path_list.append(case_dict['contrast_name_path'][t2_flair_2d_sel])
    
    # ASL, DWI, GRE
    _, asl_sel = select_ASL(case_dict['contrast_name_path'], case_dict['match_contrast_name']['ASL'])
    _, dwi_sel = select_DWI(case_dict['contrast_name_path'], case_dict['match_contrast_name']['DWI'])
    _, gre_sel = select_GRE(case_dict['contrast_name_path'], case_dict['match_contrast_name']['GRE'])
    asl_list.append(asl_sel)
    dwi_list.append(dwi_sel)
    gre_list.append(gre_sel)
    asl_path_list.append(case_dict['contrast_name_path'][asl_sel])
    dwi_path_list.append(case_dict['contrast_name_path'][dwi_sel])
    gre_path_list.append(case_dict['contrast_name_path'][gre_sel])
    
    # PET
    _, _, mac_sel, qclear_sel = select_PET(case_dict['contrast_name_path'], case_dict['match_contrast_name']['PET'])
    pet_mac_list.append(mac_sel)
    pet_qclear_list.append(qclear_sel)
    pet_mac_path_list.append(case_dict['contrast_name_path'][mac_sel])
    pet_qclear_path_list.append(case_dict['contrast_name_path'][qclear_sel])

case_0101
case_0102
case_0103
case_0104
case_0105
case_0106
case_0107
case_0108
case_0109
case_0110
case_0111
case_0112
case_0113
case_0114
case_0115
case_0116
case_0117
case_0118
case_0119
case_0120
case_0121
case_0122
case_0123
case_0124
case_0125
case_0126
case_0127
case_0128
case_0129
case_0130
case_0131
case_0132
case_0133
case_0134
case_0135
case_0136
case_0137
case_0138
case_0139
case_0140
case_0141
case_0142
case_0143
case_0144
case_0145
case_0146
case_0147
case_0148
case_0149
case_0150
case_0151
case_0152
case_0153
case_0154
case_0155
case_0156
case_0157
case_0158
case_0159
case_0160
case_0161
case_0162
case_0163
case_0164
case_0165
case_0166
case_0167
case_0168
case_0169
case_0170
case_0171
case_0172
case_0173
case_0174
case_0175
case_0176
case_0177
case_0178
case_0179
case_0180
case_0181
case_0182
case_0183
case_0184
case_0185
case_0186
case_0187
case_0188
case_0189
case_0190
case_0191
case_0192
case_0193
case_0194
case_0195
case_0196
case_0197
case_0198
case_0199
case_0200


In [24]:
df = pd.DataFrame(data={'case_id': case_id_list, 
                        'T1_GRE': t1_gre_list, 
                        'T1_SE': t1_se_list, 
                        'T1c_GRE': t1c_gre_list, 
                        'T1c_SE': t1c_se_list,
                        'T2-FLAIR': t2_flair_gre_list,
                        'T2-FLAIR_2D': t2_flair_2d_list,
                        'ASL': asl_list,
                        'DWI': dwi_list,
                        'GRE': gre_list,
                        'PET_MAC': pet_mac_list,
                        'PET_QCLEAR': pet_qclear_list,
                        'T1_GRE_path': t1_gre_path_list, 
                        'T1_SE_path': t1_se_path_list, 
                        'T1c_GRE_path': t1c_gre_path_list, 
                        'T1c_SE_path': t1c_se_path_list,
                        'T2-FLAIR_path': t2_flair_gre_path_list,
                        'T2-FLAIR_2D_path': t2_flair_2d_path_list,
                        'ASL_path': asl_path_list,
                        'DWI_path': dwi_path_list,
                        'GRE_path': gre_path_list,
                        'PET_MAC_path': pet_mac_path_list,
                        'PET_QCLEAR_path': pet_qclear_path_list
                        })

In [25]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,case_id,T1_GRE,T1_SE,T1c_GRE,T1c_SE,T2-FLAIR,T2-FLAIR_2D,ASL,DWI,GRE,PET_MAC,PET_QCLEAR,T1_GRE_path,T1_SE_path,T1c_GRE_path,T1c_SE_path,T2-FLAIR_path,T2-FLAIR_2D_path,ASL_path,DWI_path,GRE_path,PET_MAC_path,PET_QCLEAR_path
0,case_0101,9_Ax_T1_BRAVO,None,None,None,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,/data/jiahong/data/FDG_PET/case_0101/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0101/PET/1.2.8...
1,case_0102,9_Ax_T1_BRAVO,None,None,None,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,/data/jiahong/data/FDG_PET/case_0102/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0102/PET/1.2.8...
2,case_0103,12_Ax_T1_MPRAGE,None,26_Ax_T1_MPRAGE_+C,29_Ax_T1_+C,6_Sag_SPACE_FLAIR,None,15_ASL,11_Ax_DWI_ADC,9_Ax_GRE,None,13_BRAIN_Q.CLEAR,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0103/PET/1.2.8...
3,case_0104,9_Ax_T1_BRAVO,None,None,None,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,/data/jiahong/data/FDG_PET/case_0104/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0104/PET/1.2.8...
4,case_0105,9_Ax_T1_BRAVO,None,None,None,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,/data/jiahong/data/FDG_PET/case_0105/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0105/PET/1.2.8...
5,case_0106,None,None,None,None,None,2_Ax_T2_FLAIR,None,6_Ax_DWI,None,None,13_BRAIN_Q.CLEAR,None,None,None,None,None,/data/jiahong/data/FDG_PET/case_0106/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0106/MR/1.2.84...,None,None,/data/jiahong/data/FDG_PET/case_0106/PET/1.2.8...
6,case_0107,9_Ax_T1_BRAVO,None,None,None,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,/data/jiahong/data/FDG_PET/case_0107/PET/1.2.7...,None,None,None,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/PET/1.2.7...,/data/jiahong/data/FDG_PET/case_0107/PET/1.2.7...
7,case_0108,9_Ax_T1_BRAVO,None,None,None,3_COR_CUBE_FLAIR_PROMO,6_Ax_T2_FLAIR_FS,None

In [26]:
df.to_csv('series_selected_new.csv', index=False)

In [ ]:
df = pd.read_csv('series_selected_new.csv')

### check failed/wrong cases

In [231]:
case_id = 'case-0298'
case_path = os.path.join(dir_path, case_id)
case_dict = select_contrasts(case_path)

In [232]:
case_dict['contrast_name_path'].keys()

dict_keys([None, '901_SCREENSAVE', '501_Average_DC', '800_FILT_PHA_AX_T2_SWAN_WO', '2_SAG_T1_FLAIR_WO', '600_COR_T2_FRFSE_MPR_WO', '1201_COR_T1_3D_FSPGR_MPR_W', '503_T2-weighted_trace', '4_AX_DWI_EPI_WO', '602_SAG_T2_FRFSE_MPR_WO', '13_WATER_COR_T1_IDEAL_W', '900_Cerebral_Blood_Flow', '6_AX_T2_FRFSE_2MM_WO', '9_AX_3D_ASL_WO', '1200_SAG_T1_3D_FSPGR_MPR_W', '400_Apparent_Diffusion_Coefficient_mm2.per.s', '801_AX_T2_SWAN_MPR_WO', '1_LOC_3PLANE_SSFSE_WO', '7_AX_T1_SE_WO', '8_AX_T2_SWAN_WO', '801_Study_acquired_outside_hospital', '11_AX_T1_SE_W', '401_Exponential_Apparent_Diffusion_Coefficient', '12_AX_T1_3D_FSPGR_2MM_W', '10_AX_T2_FLAIR_W', '500_SCREENSAVE', '502_Exponential_attenuation', '801_PACS_nomination_form', '13_BRAIN_Q.CLEAR', '1302_PET_MIP_10', '1_PET.per.CT_Brain_Metab_IMGPT0006', '12_BRAIN_3D_NAC', '1300_PET_FUSED_AXIAL', '1301_PET_MIP_5', '1_CT_SCOUT_BRAIN', '3_CT_FUSION'])

In [233]:
for series_id in case_dict['contrast_name_path'].keys():
    try:
        dicom_path = glob.glob(case_dict['contrast_name_path'][series_id]+'/*')[0]
        ds = pydicom.filereader.dcmread(dicom_path)
    #     if ds.Modality == 'PT':
    #         print(series_id, ds.SeriesDescription, ds.Modality)
        if 'DWI' in ds.SeriesDescription:
            print(series_id, ds.SeriesDescription)
#         if 'GRE' in ds.SeriesDescription:
#             print(series_id, ds.SeriesDescription)
#         if 'FLAIR' in ds.SeriesDescription:
#             print(series_id, ds.SeriesDescription)
#         if 'T1' in ds.SeriesDescription:
#             print(series_id, ds.SeriesDescription)
    except:
        pass

4_AX_DWI_EPI_WO AX DWI EPI WO


In [234]:
series_id = '4_AX_DWI_EPI_WO'
print(case_dict['contrast_name_path'][series_id])
dicom_path = glob.glob(case_dict['contrast_name_path'][series_id]+'/*')[0]
ds = pydicom.filereader.dcmread(dicom_path)

/data/jiahong/data/FDG_PET/case-0298/MR/4_AX_DWI_EPI_WO


In [93]:
ds

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 194
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Positron Emission Tomography Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.840.113619.2.5.31139203.27539.1465253160.873
(0002, 0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.5.4
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_354'
-------------------------------------------------
(0008, 0000) Group Length                        UL: 668
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0012) Instance Creation Date              DA: '20160527'
(0008, 0013) Instance Creation Time              TM: '112814'
(0008, 0014) Instance C

In [85]:
series_id

In [83]:
ds.SeriesDescription 

'Ax DWI'

### check revised list

In [235]:
df_checked = pd.read_csv('series_selected_new_checked.csv')

In [256]:
contrast_name_list = ['T1_GRE', 'T1_SE', 'T1c_GRE', 'T1c_SE', 'T2-FLAIR', 'T2-FLAIR_2D', 
                 'ASL', 'DWI', 'GRE', 'PET_MAC', 'PET_QCLEAR', 'PET_TOF']

for contrast_name in contrast_name_list:
    x = df_checked[contrast_name]
    y = df_checked[contrast_name+'_path']
    print(contrast_name)
#     print(df_checked['case_id'].loc[(x.isna()) & (y!='None')])
#     print(df_checked['case_id'].loc[(~x.isna()) & (y=='None')])
    print(df_checked['case_id'].loc[~x.isna()].shape)
#     df_checked[contrast_name+'_path'].loc[x.isna()] = 'None'

T1_GRE
(241,)
T1_SE
(82,)
T1c_GRE
(131,)
T1c_SE
(118,)
T2-FLAIR
(230,)
T2-FLAIR_2D
(161,)
ASL
(192,)
DWI
(254,)
GRE
(220,)
PET_MAC
(202,)
PET_QCLEAR
(171,)
PET_TOF
(6,)


In [264]:
df_checked = df_checked.rename(columns={'T2-FLAIR':'T2_FLAIR', 'T2-FLAIR_2D':'T2_FLAIR_2D',
                                       'T2-FLAIR_path':'T2_FLAIR_path', 'T2-FLAIR_2D_path':'T2_FLAIR_2D_path'})

In [266]:
df_checked.to_csv('series_selected_new_checked.csv', index=False)

In [265]:
df_checked

,case_id,T1_GRE,T1_SE,T1c_GRE,T1c_SE,T2_FLAIR,T2_FLAIR_2D,ASL,DWI,GRE,PET_MAC,PET_QCLEAR,PET_TOF,T1_GRE_path,T1_SE_path,T1c_GRE_path,T1c_SE_path,T2_FLAIR_path,T2_FLAIR_2D_path,ASL_path,DWI_path,GRE_path,PET_MAC_path,PET_QCLEAR_path,PET_TOF_path
0,case_0101,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,NaN,/data/jiahong/data/FDG_PET/case_0101/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0101/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0101/PET/1.2.8...,None
1,case_0102,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,NaN,/data/jiahong/data/FDG_PET/case_0102/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0102/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0102/PET/1.2.8...,None
2,case_0103,12_Ax_T1_MPRAGE,NaN,26_Ax_T1_MPRAGE_+C,29_Ax_T1_+C,6_Sag_SPACE_FLAIR,NaN,15_ASL,11_Ax_DWI_TRACEW,9_Ax_GRE,NaN,13_BRAIN_Q.CLEAR,NaN,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0103/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0103/PET/1.2.8...,None
3,case_0104,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,NaN,/data/jiahong/data/FDG_PET/case_0104/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0104/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0104/PET/1.2.8...,None
4,case_0105,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,NaN,/data/jiahong/data/FDG_PET/case_0105/PET/1.2.8...,None,None,None,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0105/PET/1.2.8...,/data/jiahong/data/FDG_PET/case_0105/PET/1.2.8...,None
5,case_0106,700_AX_Processed_pre,NaN,800_AX_Processed_post,NaN,NaN,2_Ax_T2_FLAIR,NaN,6_Ax_DWI,NaN,NaN,13_BRAIN_Q.CLEAR,NaN,/data/jiahong/data/FDG_PET/case_0106/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0106/MR/1.2.84...,None,None,/data/jiahong/data/FDG_PET/case_0106/MR/1.2.84...,None,/data/jiahong/data/FDG_PET/case_0106/MR/1.2.84...,None,None,/data/jiahong/data/FDG_PET/case_0106/PET/1.2.8...,None
6,case_0107,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,11_Ax_T2_FLAIR_FS_ARC,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear,NaN,/data/jiahong/data/FDG_PET/case_0107/PET/1.2.7...,None,None,None,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/data/jiahong/data/FDG_PET/case_0107/MR/1.2.84...,/dat

### convert selected dicoms into nifti

In [270]:
# copy all selected dicom series
save_path = '/data/jiahong/data/FDG_PET_preprocessed'
contrast_name_list = [ ]


for row_idx in range(df_checked.shape[0]):
    case_id = df_checked['case_id'].iloc[row_idx]
    save_case_path = os.path.join(save_path, case_id)
    if not os.path.exists(save_case_path):
        os.mkdir(save_case_path)

    for contrast_name in contrast_name_list:
        contrast_path = df_checked[contrast_name+'_path'].iloc[row_idx]
        save_contrast_path = os.path.join(save_case_path, contrast_name)
        if contrast_path == 'None' or os.path.exists(save_contrast_path):
            continue
        shutil.copytree(contrast_path, save_contrast_path)
    print('Finished', case_id)

Finished case_0101
Finished case_0102
Finished case_0103
Finished case_0104
Finished case_0105
Finished case_0106
Finished case_0107
Finished case_0108
Finished case_0109
Finished case_0110
Finished case_0111
Finished case_0112
Finished case_0113
Finished case_0114
Finished case_0115
Finished case_0116
Finished case_0117
Finished case_0118
Finished case_0119
Finished case_0120
Finished case_0121
Finished case_0122
Finished case_0123
Finished case_0124
Finished case_0125
Finished case_0126
Finished case_0127
Finished case_0128
Finished case_0129
Finished case_0130
Finished case_0131
Finished case_0132
Finished case_0133
Finished case_0134
Finished case_0135
Finished case_0136
Finished case_0137
Finished case_0138
Finished case_0139
Finished case_0140
Finished case_0141
Finished case_0142
Finished case_0143
Finished case_0144
Finished case_0145
Finished case_0146
Finished case_0147
Finished case_0148
Finished case_0149
Finished case_0150
Finished case_0151
Finished case_0152
Finished cas

In [272]:
# selected dicom to nifti
for row_idx in range(df_checked.shape[0]):
    case_id = df_checked['case_id'].iloc[row_idx]
    save_case_path = os.path.join(save_path, case_id)

    for contrast_name in contrast_name_list:
        save_contrast_path = os.path.join(save_case_path, contrast_name)
        if not os.path.exists(save_contrast_path) or os.path.exists(os.path.join(save_case_path, contrast_name+'.nii')):
            continue
        command='dcm2niix -f '+ contrast_name + ' -o '+ save_case_path +' '+ save_contrast_path
        os.system(command)
        
    print('Finished', case_id)

Finished case_0101
Finished case_0102
Finished case_0103
Finished case_0104
Finished case_0105
Finished case_0106
Finished case_0107
Finished case_0108
Finished case_0109
Finished case_0110
Finished case_0111
Finished case_0112
Finished case_0113
Finished case_0114
Finished case_0115
Finished case_0116
Finished case_0117
Finished case_0118
Finished case_0119
Finished case_0120
Finished case_0121
Finished case_0122
Finished case_0123
Finished case_0124
Finished case_0125
Finished case_0126
Finished case_0127
Finished case_0128
Finished case_0129
Finished case_0130
Finished case_0131
Finished case_0132
Finished case_0133
Finished case_0134
Finished case_0135
Finished case_0136
Finished case_0137
Finished case_0138
Finished case_0139
Finished case_0140
Finished case_0141
Finished case_0142
Finished case_0143
Finished case_0144
Finished case_0145
Finished case_0146
Finished case_0147
Finished case_0148
Finished case_0149
Finished case_0150
Finished case_0151
Finished case_0152
Finished cas

In [273]:
# check failed
failed_list = []
for row_idx in range(df_checked.shape[0]):
    case_id = df_checked['case_id'].iloc[row_idx]
    save_case_path = os.path.join(save_path, case_id)

    for contrast_name in contrast_name_list:
        save_contrast_path = os.path.join(save_case_path, contrast_name)
        if os.path.exists(save_contrast_path) and not os.path.exists(os.path.join(save_case_path, contrast_name+'.nii')):
            failed_list.append([case_id, contrast_name])
    
print(failed_list)

[['case_0178', 'T1c_GRE'], ['case_0178', 'T2_FLAIR']]


In [ ]:
~
~
~
"ASL.json" 45L, 1260C                                                                                                 1,1           All
{
        "Modality": "MR",
        "MagneticFieldStrength": 3,
        "ImagingFrequency": 127.77,
        "ManufacturersModelName": "SIGNA_PET_MR",
        "DeviceSerialNumber": "0000415723SHMR15",
        "StationName": "SECTRA_XD_07",
        "BodyPartExamined": "BRAIN",
        "PatientPosition": "HFS",
        "ProcedureStepDescription": "MR_Brain_wo_Seizure",
        "SoftwareVersions": "27_LX_MR_Software_release:MP26.0_R01_1824.b",
        "MRAcquisitionType": "3D",
        "SeriesDescription": "3D_ASL_color",
        "ProtocolName": "_1PET_MRI_EPILEPSY",
        "ScanningSequence": "RM",
        "SequenceVariant": "NONE",
        "ScanOptions": "EDR_GEMS_SPIRAL_GEMS",
        "ImageType": ["DERIVED", "PRIMARY", "ASL", "PERFUSION", "ASL"],
        "RawImage": false,
        "SeriesNumber": 16,
        "AcquisitionTime": "13:12:11.000000",
        "AcquisitionNumber": 1,
        "SliceThickness": 4,
        "SpacingBetweenSlices": 4,
        "SAR": 0.429966,
        "EchoTime": 0.010728,
        "RepetitionTime": 4.855,
        "InversionTime": 2.025,
        "FlipAngle": 111.1,
        "CoilString": "8HRBRAIN",
        "PercentPhaseFOV": 100,
        "AcquisitionMatrixPE": 8,
        "ReconMatrixPE": 128,
        "PixelBandwidth": 976.562,
        "ImageOrientationPatientDICOM": [
                1,
                -0,
                0,
                -0,
                1,
                0       ],
        "InPlanePhaseEncodingDirectionDICOM": "ROW",
        "ConversionSoftware": "dcm2niix",
        "ConversionSoftwareVersion": "v1.0.20181125  (JP2:OpenJPEG) GCC9.3.0"
}


~
~
"ASLa.json" 44L, 1221C                                                                                                1,1           All
{
        "Modality": "MR",
        "MagneticFieldStrength": 3,
        "ImagingFrequency": 127.77,
        "ManufacturersModelName": "SIGNA_PET_MR",
        "DeviceSerialNumber": "0000415723SHMR15",
        "StationName": "SECTRA_XD_07",
        "BodyPartExamined": "BRAIN",
        "PatientPosition": "HFS",
        "ProcedureStepDescription": "MR_Brain_wo_Seizure",
        "SoftwareVersions": "27_LX_MR_Software_release:MP26.0_R01_1824.b",
        "MRAcquisitionType": "3D",
        "SeriesDescription": "3D_ASL_color",
        "ProtocolName": "_1PET_MRI_EPILEPSY",
        "ScanningSequence": "RM",
        "SequenceVariant": "NONE",
        "ScanOptions": "EDR_GEMS_SPIRAL_GEMS",
        "ImageType": ["ORIGINAL", "PRIMARY", "ASL"],
        "SeriesNumber": 16,
        "AcquisitionTime": "13:12:11.000000",
        "AcquisitionNumber": 1,
        "SliceThickness": 4,
        "SpacingBetweenSlices": 4,
        "SAR": 0.429966,
        "EchoTime": 0.010728,
        "RepetitionTime": 4.855,
        "InversionTime": 2.025,
        "FlipAngle": 111.1,
        "CoilString": "8HRBRAIN",
        "PercentPhaseFOV": 100,
        "AcquisitionMatrixPE": 8,
        "ReconMatrixPE": 128,
        "PixelBandwidth": 976.562,
        "ImageOrientationPatientDICOM": [
                1,
                -0,
                0,
                -0,
                1,
                0       ],
        "InPlanePhaseEncodingDirectionDICOM": "ROW",
        "ConversionSoftware": "dcm2niix",
        "ConversionSoftwareVersion": "v1.0.20181125  (JP2:OpenJPEG) GCC9.3.0"
}

In [ ]:
def generate_nii(i):
    
    resetdir(i+'_nii')
    command='dcm2niix -o  '+i+'_nii'+' '+i
    os.system(command)
    nii_folders=[]
    for j in listdir(i+'_nii'):
        if 'nii' in j.split('/')[-1]:
            nii_folders.append(j)

    if len(nii_folders)!=1:
        raise Exception("Sorry, no numbers below zero")
    else:
        nii_path=nii_folders[0]

    shutil.move(nii_path,i+'.nii')
    shutil.rmtree(i+'_nii')

### draft

In [46]:
PET_mac_list = []
PET_qclear_list = []
for case_id in case_id_list:
    case_path = os.path.join(dir_path, case_id)
    case_dict = select_contrasts(case_path)
    _, _, mac_sel, qclear_sel = select_PET(case_dict['contrast_name_path'], case_dict['match_contrast_name']['PET'])
    print(case_id, mac_sel, '\t', qclear_sel)
    PET_mac_list.append(mac_sel)
    PET_qclear_list.append(qclear_sel)

case_0101 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0102 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0103 None 	 13_BRAIN_Q.CLEAR
case_0104 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0105 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0106 None 	 13_BRAIN_Q.CLEAR
case_0107 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0108 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0109 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0110 None 	 13_BRAIN_Q.CLEAR
case_0111 14_Static_3D_MAC 	 15_Static_3D_Q.Clear
case_0112 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0113 None 	 14_BRAIN_Q.CLEAR
case_0114 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0115 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0116 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0117 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0118 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0119 4_Static_3D_MAC 	 5_Static_3D_Q.Clear
case_0120 None 	 None
case_0121 6_Static_3D_MAC 	 7_Static_3D_Q.Clear
case_0122 None 	 None
case_0123 None 	 12_BRAIN_Q_CLEAR
case_0124 None 	

In [41]:
asl_list = []
dwi_list = []
gre_list = []
for case_id in case_id_list:
    case_path = os.path.join(dir_path, case_id)
    case_dict = select_contrasts(case_path)
    _, asl_sel = select_ASL(case_dict['contrast_name_path'], case_dict['match_contrast_name']['ASL'])
    _, dwi_sel = select_DWI(case_dict['contrast_name_path'], case_dict['match_contrast_name']['DWI'])
    _, gre_sel = select_GRE(case_dict['contrast_name_path'], case_dict['match_contrast_name']['GRE'])
    print(case_id, asl_sel, dwi_sel, gre_sel)
    asl_list.append(asl_sel)
    dwi_list.append(dwi_sel)
    gre_list.append(gre_sel)

case_0101 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0102 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0103 15_ASL 11_Ax_DWI_ADC 9_Ax_GRE
case_0104 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0105 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0106 None 6_Ax_DWI None
case_0107 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0108 None 7_Ax_DWI 8_AX_GRE
case_0109 None None None
case_0110 8_3D_ASL_color 6_Ax_DWI_3mm 5_AX_GRE
case_0111 4_AX_3D_ASL_color 7_Ax_DWI+C 6_AX_GRE_+C
case_0112 None None None
case_0113 None None None
case_0114 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0115 None None None
case_0116 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0117 15_3D_ASL_color 11_Ax_DWI 12_AX_GRE
case_0118 10_3D_ASL_color 13_Ax_DWI 11_AX_GRE
case_0119 16_3D_ASL_color 15_Ax_DWI 14_AX_GRE
case_0120 None None None
case_0121 17_3D_ASL_color 16_Ax_DWI 15_AX_GRE
case_0122 17_3D_ASL_color 16_Ax_DWI 15_AX_GRE
case_0123 None 5_Ax_DWI:_Multi-bValue None
case_0124 3_3D_ASL_color 11_Ax_DWI 12_AX_GRE
case_0125 16_3D_ASL_color 15_Ax_DWI

In [15]:
t2_flair_gre_list = []
for case_id in case_id_list:
    case_path = os.path.join(dir_path, case_id)
    case_dict = select_contrasts(case_path)
    _, t2_flair_gre_sel = select_T2_FLAIR(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T2-FLAIR'])
    print(case_id, t2_flair_gre_sel)
    t2_flair_gre_list.append(t2_flair_gre_sel)

case_0101 10_COR_T2_CUBE_FLAIR
case_0102 10_COR_T2_CUBE_FLAIR
case_0103 6_Sag_SPACE_FLAIR
No SeriesDescription /data/jiahong/data/FDG_PET/case_0104/MR/1.2.840.114350.2.171.2.798268.2.607387237.1/0_20190529/31304060_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0104/PET/1.2.840.114350.2.171.2.798268.2.607387236.1/0_20190529/31530107_/I0000.dcm
case_0104 10_COR_T2_CUBE_FLAIR
case_0105 10_COR_T2_CUBE_FLAIR
No SeriesDescription /data/jiahong/data/FDG_PET/case_0106/MR/1.2.840.114350.2.171.2.798268.2.618289956.1/0_20190206/501_/I0000.dcm
case_0106 None
No SeriesDescription /data/jiahong/data/FDG_PET/case_0107/PET/1.2.752.24.7.1532735313.6473132/0_20190607/32936791_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0107/PET/1.2.752.24.7.1532735313.6473132/0_20190607/32936787_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0107/PET/1.2.752.24.7.1532735313.6473132/0_20190607/32936776_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0

In [8]:
t1_gre_list = []
t1_se_list = []
t1c_gre_list = []
t1c_se_list = []
for case_id in case_id_list:
    case_path = os.path.join(dir_path, case_id)
    case_dict = select_contrasts(case_path)
    _, _, t1_gre_sel, t1_se_sel = select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1'])
    _, _, t1c_gre_sel, t1c_se_sel = select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1c'])
    print(case_id, t1_gre_sel, '\t', t1_se_sel, '\t', t1c_gre_sel, '\t', t1c_se_sel)
    t1_gre_list.append(t1_gre_sel)
    t1_se_list.append(t1_se_sel)
    t1c_gre_list.append(t1c_gre_sel)
    t1c_se_list.append(t1c_se_sel)

case_0101 9_Ax_T1_BRAVO 	 None 	 None 	 None
case_0102 9_Ax_T1_BRAVO 	 None 	 None 	 None
case_0103 12_Ax_T1_MPRAGE 	 None 	 26_Ax_T1_MPRAGE_+C 	 29_Ax_T1_+C
No SeriesDescription /data/jiahong/data/FDG_PET/case_0104/MR/1.2.840.114350.2.171.2.798268.2.607387237.1/0_20190529/31304060_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0104/PET/1.2.840.114350.2.171.2.798268.2.607387236.1/0_20190529/31530107_/I0000.dcm
case_0104 9_Ax_T1_BRAVO 	 None 	 None 	 None
case_0105 9_Ax_T1_BRAVO 	 None 	 None 	 None
No SeriesDescription /data/jiahong/data/FDG_PET/case_0106/MR/1.2.840.114350.2.171.2.798268.2.618289956.1/0_20190206/501_/I0000.dcm
case_0106 None 	 None 	 None 	 None
No SeriesDescription /data/jiahong/data/FDG_PET/case_0107/PET/1.2.752.24.7.1532735313.6473132/0_20190607/32936791_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0107/PET/1.2.752.24.7.1532735313.6473132/0_20190607/32936787_/I0000.dcm
No SeriesDescription /data/jiahong/data/FDG_PET/case_0107/PET/

In [53]:
case_dict

{'contrast_name_path': {'1401': '/data/jiahong/data/FDG_PET/30804_11012018/1401',
  '1101': '/data/jiahong/data/FDG_PET/30804_11012018/1101',
  '007': '/data/jiahong/data/FDG_PET/30804_11012018/007',
  '801': '/data/jiahong/data/FDG_PET/30804_11012018/801',
  '006': '/data/jiahong/data/FDG_PET/30804_11012018/006',
  '800': '/data/jiahong/data/FDG_PET/30804_11012018/800',
  '009': '/data/jiahong/data/FDG_PET/30804_11012018/009',
  '5350': '/data/jiahong/data/FDG_PET/30804_11012018/5350',
  '5353': '/data/jiahong/data/FDG_PET/30804_11012018/5353',
  '401': '/data/jiahong/data/FDG_PET/30804_11012018/401',
  '013': '/data/jiahong/data/FDG_PET/30804_11012018/013',
  '11776': '/data/jiahong/data/FDG_PET/30804_11012018/11776',
  '400': '/data/jiahong/data/FDG_PET/30804_11012018/400',
  '008': '/data/jiahong/data/FDG_PET/30804_11012018/008',
  '010': '/data/jiahong/data/FDG_PET/30804_11012018/010',
  '901': '/data/jiahong/data/FDG_PET/30804_11012018/901',
  '1100': '/data/jiahong/data/FDG_PET/

In [12]:
for case_id in case_id_list_num:
    case_path = os.path.join(dir_path, case_id)
    case_dict = select_contrasts(case_path)
    print(case_id)
    print(select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1']))
    print(select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1c']))
 

0_05102019
(['009', '017', '900', '901'], [], '009', None)
([], [], None, None)
842_06152015
(['007', '013'], [], '007', None)
([], [], None, None)
852_06182015
(['006', '012'], [], '006', None)
(['015'], ['016'], '015', '016')
869_06252015
(['007'], [], '007', None)
(['014'], ['016'], '014', '016')
1496_05272016
(['007', '010'], [], '007', None)
(['022'], ['023'], '022', '023')
1498_05312016
(['007'], [], '007', None)
(['016'], ['018'], '016', '018')
1512_06062016
(['006'], [], '006', None)
(['013'], ['017'], '013', '017')
1549_06232016
(['007'], [], '007', None)
(['014'], ['016'], '014', '016')
1559_06282016
(['009'], [], '009', None)
(['017'], ['019'], '017', '019')
1572_07052016
(['009'], [], '009', None)
(['018'], ['020'], '018', '020')
1604_07142016
(['007'], [], '007', None)
(['016'], ['018'], '016', '018')
1610_07152016
(['008'], [], '008', None)
(['018'], ['020'], '018', '020')
1619_07192016
(['008'], [], '008', None)
(['017'], ['019'], '017', '019')
1657_08022016
(['012'], []

In [ ]:
check list
2010_02212017

In [9]:
len(t1_gre_list)

280

In [49]:
df = pd.DataFrame(data={'case_id': case_id_list, 
                        'T1_GRE': t1_gre_list, 
                        'T1_SE': t1_se_list, 
                        'T1c_GRE': t1c_gre_list, 
                        'T1c_SE': t1c_se_list,
                        'T2-FLAIR': t2_flair_gre_list,
                        'ASL': asl_list,
                        'DWI': dwi_list,
                        'GRE': gre_list,
                        'PET_MAC': PET_mac_list,
                        'PET_QCLEAR': PET_qclear_list
                        })

In [50]:
df.to_csv('series_selected_new.csv', index=False)

In [51]:
df = pd.read_csv('series_selected_new.csv')

In [52]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,case_id,T1_GRE,T1_SE,T1c_GRE,T1c_SE,T2-FLAIR,ASL,DWI,GRE,PET_MAC,PET_QCLEAR
0,case_0101,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear
1,case_0102,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear
2,case_0103,12_Ax_T1_MPRAGE,NaN,26_Ax_T1_MPRAGE_+C,29_Ax_T1_+C,6_Sag_SPACE_FLAIR,15_ASL,11_Ax_DWI_ADC,9_Ax_GRE,NaN,13_BRAIN_Q.CLEAR
3,case_0104,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear
4,case_0105,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear
5,case_0106,NaN,NaN,NaN,NaN,NaN,NaN,6_Ax_DWI,NaN,NaN,13_BRAIN_Q.CLEAR
6,case_0107,9_Ax_T1_BRAVO,NaN,NaN,NaN,10_COR_T2_CUBE_FLAIR,16_3D_ASL_color,15_Ax_DWI,14_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear
7,case_0108,9_Ax_T1_BRAVO,NaN,NaN,NaN,3_COR_CUBE_FLAIR_PROMO,NaN,7_Ax_DWI,8_AX_GRE,4_Static_3D_MAC,5_Static_3D_Q.Clear
8,case_0109,9_Ax_T1_BRAVO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4_Static_3D_MAC,5_Static_3D_Q.Clear
9,case_0110,7_AX_BRAVO,NaN,20_AX_BRAVO_+C,19_AX_T1_+C,4_Sag_CUBE_FLAIR_PROMO,8_3D_ASL_color,6_Ax_DWI_3mm,5_AX_GRE,NaN,13_BRAIN_Q.CLEAR


In [111]:
path = '/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 11776/'
escaped_directory_path = glob.escape(path)

# Create the glob pattern to match files within the escaped directory path
pattern = f"{escaped_directory_path}/*"

glob.glob(pattern)

['/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 11776/IM-0001-0082-0001.dcm',
 '/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 11776/IM-0001-0073-0001.dcm',
 '/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 11776/IM-0001-0099-0001.dcm',
 '/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 11776/IM-0001-0056-0001.dcm',
 '/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 11776/IM-0001-0079-0001.dcm',
 '/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Std FTB [ %- 26.6 - 18.4 - 55.0 ] [Ranges- min - 1 - 1.556 - max ] (RGB) - 1177

In [139]:
case_path = os.path.join(dir_path, 'Fdg_Stanford_025')
# for paths in os.walk(case_path):
#     if len(paths[-1]) == 0:   # no file, go into subfolders
#         continue
#     is_dicom = False
#     for path in paths[-1]: # check whether dcm files
#         if '.dcm' in path:
#             is_dicom = True
#             break
#     if not is_dicom:
#         continue

#     contrast_path = paths[0]
#     contrast_name = os.path.basename(paths[0])
#     contrast_name_path[contrast_name] = contrast_path

# #     print(contrast_path)
#     escaped_contrast_path = glob.escape(contrast_path)
#     pattern = f"{escaped_contrast_path}/*"
#     if (not os.path.exists(escaped_contrast_path)) or len(glob.glob(pattern))==0:
#         print(contrast_path)
#         continue
    
        
case_dict = {}
contrast_name_path = {}
match_contrast_name = {'T1':[], 'T1c':[], 'T2-FLAIR':[], 'ASL':[], 'PET':[], 'T2':[], 'DWI':[], 'GRE':[]}
for paths in os.walk(case_path):
    if len(paths[-1]) == 0:   # no file, go into subfolders
        continue
    is_dicom = False
    for path in paths[-1]: # check whether dcm files
        if '.dcm' in path:
            is_dicom = True
            break
    if not is_dicom:
        continue

    # save all contrast name and path
    contrast_path = paths[0]
    contrast_name = os.path.basename(paths[0])
    contrast_name_path[contrast_name] = contrast_path

    contrast_path = glob.escape(contrast_path)
    pattern = f"{contrast_path}/*"
    if (not os.path.exists(contrast_path)) or len(glob.glob(pattern))==0:
        continue

    dicom_path = glob.glob(contrast_path+'/*')[0]
    print(dicom_path)

/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/SAG BRAVO+C - 1100/IM-0001-0082-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/BRAIN TOF 690 - 6/IM-0001-0040-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Sag T1 FLAIR - 3/IM-0001-0026-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/SAG BRAVO+C - 1400/IM-0001-0082-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/AX T2 - 10/IM-0001-0026-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/AX BRAVO - 8/IM-0001-0082-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/SCOUT - 1/IM-0001-0002-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/COR BRAVO pre - 801/IM-0001-0082-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/CT FUSION BRAIN - 3/IM-0001-0040-0001.dcm
/data/jiahong/data/FDG_PET/Fdg_Stanford_025/Mr Brain W-Wo Tumor/Sag CUBE FLAIR 

In [46]:
path=glob.glob("/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517"+'/*')[28]

In [48]:
path

'/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/11049_Std_FTB_[_%:_19.7_-_23.1_-_57.2_]_[Ranges:_min_-_1_-_1.556_-_max_'

In [47]:
glob.glob(path+'/*')

[]

In [42]:
glob.glob("/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517"+'/*')

['/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/9_Ax_GRE',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/5359_3D_Lab:_Histogram',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/1_',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/13_Ax_T1_MPRAGE_MPR_Sag_Reformat',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/2442_RAPID_AIF_VOF_Location',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/19_Ax_T1_MAP_FA_10',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/1_20190526_15H37M',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350.2.171.2.798268.2.570505336.1/0_20190517/20_Ax_T1_MAP_FA_15',
 '/data/jiahong/data/FDG_PET/case_0103/MR/1.2.840.114350

In [ ]:
case_id = case_id_list[0]
case_path = os.path.join(dir_path, case_id)
case_dict = select_contrasts(case_path)

In [25]:
t1_gre_list, t1_se_list, t1_gre_sel, t1_se_sel = select_T1_T1c(case_dict['contrast_name_path'], case_dict['match_contrast_name']['T1'])

In [26]:
t1_gre_sel, t1_se_sel

('9_Ax_T1_BRAVO', None)

In [16]:
case_dict['match_contrast_name']['T1']

['9_Ax_T1_BRAVO',
 '901_COR_T1_BRAVO',
 '900_SAG_T1_BRAVO',
 '17_SAG_T1_IR_SPGR_Dsuite_Neuroquant_PACS',
 '9_Ax_T1_BRAVO']

In [12]:
contrast_name_path['9_Ax_T1_BRAVO']

'/data/jiahong/data/FDG_PET/case_0101/PET/1.2.840.114350.2.171.2.798268.2.605226077.1/0_20190510/9_Ax_T1_BRAVO'

In [ ]:
9_Ax_T1_BRAVO
/data/jiahong/data/FDG_PET/case_0101/MR/1.2.840.114350.2.171.2.798268.2.605682975.1/0_20190510/9_Ax_T1_BRAVO

9_Ax_T1_BRAVO
/data/jiahong/data/FDG_PET/case_0101/PET/1.2.840.114350.2.171.2.798268.2.605226077.1/0_20190510/9_Ax_T1_BRAVO